In [ ]:
#!/usr/local/bin/python3
# -*- coding:UTF-8 -*-
from __future__ import absolute_import, division, print_function, unicode_literals
import pickle
import argparse
import torch
import pyaudio
from pathlib import Path
from boto3 import client
from gtts import gTTS
import speech_recognition as sr
import pygame
from io import BytesIO
import threading
import time
import wave
import urllib3
import sys
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bayes import BayesianFilter
from model import KoBertTokenizer
import tensorflow as tf
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
import logging
import unicodedata
from shutil import copyfile
from keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Hannanum
import day
intent_bf = BayesianFilter()
IntentList = ['위치','절차','번호','일정','확진자','정보']
call_bot = ''
max_len = 88
SEQ_LEN = max_len
label_list = []
token_list = []
text = ''
data = pd.read_csv('병원 질의 데이터베이스 - intent_ 절차.csv')
location_data = pd.read_csv('위치_대답.csv')
han = Hannanum()
train = pd.read_csv("train_data", names=['src', 'tar'], sep="\t")
train = train.reset_index()
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
index_to_ner = {i:j for j, i in label_dict.items()}
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
def create_model():
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)
    token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
    mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋
    bert_outputs = model([token_inputs, mask_inputs])
    bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
    nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
    nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
    nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
    return nr_model
nr_model = create_model()
nr_model.load_weights('model_weight.h5')
def ner_inference(test_sentence):
    tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, pad_to_max_length=True)])
    tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
    ans = nr_model.predict([tokenized_sentence, tokenized_mask])
    ans = np.argmax(ans, axis=2)
    label_list = []
    token_list = []
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, ans[0]):
        if (token.startswith("▁")):
            new_labels.append(index_to_ner[label_idx])
            new_tokens.append(token[1:])
        elif (token=='[CLS]'):
            pass
        elif (token=='[SEP]'):
            pass
        elif (token=='[PAD]'):
            pass
        elif (token != '[CLS]' or token != '[SEP]'):
            new_tokens[-1] = new_tokens[-1] + token
    for token, label in zip(new_tokens, new_labels):
        label_list.append(label)
        token_list.append(token)
    return label_list, token_list
def init_fuction():
    pygame.init()
    # python version check python 2.0 보다 낮을 경우 sys reload
    if sys.version_info <= (2,7):
        reload(sys)
        sys.setdefaultencoding('utf-8')
    for idx in range(data.shape[0]):
        intent_bf.fit(data['Question'][idx],data['Intent'][idx])
def callback(recognizer, audio):
    global call_bot
    try:
        call_bot = recognizer.recognize_google(audio, language='ko')
        # recognizer.recognize_google = Google Web Speech Application Programming Interface
        print("you said : " + call_bot)
    except sr.UnknownValueError:
        TTS('알아 듣지 못했습니다. 다시 한 번 말해주세요.')
    except sr.RequestError as e:
        TTS('리퀘스트 에러입니다. 에러는 {0}입니다.'.format(e))
def background_call():
    global call_bot
    print('Speak plz : ')
    r = sr.Recognizer()
    m = sr.Microphone()
    with m as source:
        r.adjust_for_ambient_noise(source)
    stop_listening = r.listen_in_background(m, callback)
    for _ in range(30):
        time.sleep(0.1)
    stop_listening(wait_for_stop = False)
    # 계속해서 듣는게 아니라 while 문을 통해서 도는 것과 같이 동작하는 방식.
def STT():
    global text
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('Speak Anything :')
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language='ko-KR')
            print('You said : {}'.format(text))
        except:
            print('Sirri could not recignize your voice')
    return text
def TTS(text):
    tts = gTTS(text=text, lang='ko')
    fp = BytesIO()
    tts.write_to_fp(fp)
    fp.seek(0)
    pygame.mixer.init()
    pygame.mixer.music.load(fp)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)
def main():
    init_fuction()
    while True:
        try:
            background_call()
            while '바보' in call_bot:
                text = STT()
                department_list = []
                intent, score = intent_bf.predict(text)
                for l in IntentList:
                    if intent == l:
                        c = IntentList.index(l)
                elif c==1: #절차
                    print('절차')
                    if '예약' in text:
                        TTS('054-260-8188로 전화하면 돼요')
                    elif '접수' in text:
                        TTS('3일전 부터 육류 섭취와 음주 과로를 피하고 문진표와 채변을 제출하시면 돼요')
                    elif '유의' in text:
                        TTS('건강검진 후에는 술과 담배, 매운 음식을 자제하고 수면 내시경 고객은 당일 운전을 금하셔야 해요')
                    elif '절차' in text:
                        TTS('예약, 접수, 탈의, 검사진행, 검사종료 후 결과 상담으로 진행 돼요')
                
                elif '종료' in text:
                    TTS('종료합니다.')
                    break
                else :
                    TTS('다시 말씀해 주시겠어요?')
                text = ''
        except KeyboardInterrupt:
            print('Ctrl + C 중지 ')
            break
# main()

In [177]:
pos_dict = {
    'Adverb': {'건강검진', '예약', '유의', '주의', '진료절차', 
               '접수', '예약시간', '입원', '절차', '퇴원'}
}
from soynlp.postagger import Dictionary
from soynlp.postagger import LRTemplateMatcher
from soynlp.postagger import LREvaluator
from soynlp.postagger import SimpleTagger
from soynlp.postagger import UnknowLRPostprocessor
tagger = SimpleTagger(LRTemplateMatcher(Dictionary(pos_dict)), LREvaluator(), UnknowLRPostprocessor())

sent = '저기 혹시 건강검진 하려는데 어떻게 해요'
token_list = tagger.tag(sent)
TTS(token_list)       
word1 = ''
list_word = []
for ele in token_list:
    if ele[1] == 'Adverb':
        word, adverb = ele
        list_word.append(word)
     
    
if '건강검진' in list_word:
    if '건강검진' or '예약' or '접수' in list_word:
        TTS("054에 260에 8188로 전화하면 돼요")
    elif '유의' or '주의' in list_word:
        TTS("건강검진 후에는 술과 담배, 매운 음식을 자제하고 수면 내시경 고객은 당일 운전을 금하셔야 해요")
    elif '진료' or '절차' in list_word:
        TTS("예약, 접수, 탈의, 검사진행, 검사종료 후 결과 상담으로 진행 돼요")
    else:
        TTS("조금만 풀어서 설명해주시겠어요")
if '접수' in list_word: 
    if '접수' or '예약' or '절차' in list_word:
        TTS("인터넷 접수하거나 054-260-8001로 전화 주세요")
    elif '예약시간' in list_word:
         TTS("검진예약시간은 평일 오전 8시 부터 오후 5시까지이고 토요일엔 오전 8시부터 정오 12시까지에요")
    elif '유의' or '주의' in list_word:
         TTS("예약 시간을 지키고 금식을 유지해주세요")
    else:
        TTS("조금만 풀어서 설명해주시겠어요")
elif '입원' in list_word:
    if '입원' or '절차' in list_word:
        TTS("입원 약정서를 작성한 후 입원 결정서가 제출 되면 입실이 진행돼요")
    else:
        TTS("조금만 풀어서 설명해주시겠어요")
elif '퇴원' in list_word:
    if '퇴원' or '절차' in list_word:
        TTS('퇴원 수속 안내문과 입원 진료비를 납부 한 후 간호사의 안내에 따라 귀가 하시면 돼요')
    else:
        TTS("조금만 풀어서 설명해주시겠어요")

else:
    TTS("조금만 풀어서 설명해주시겠어요")


[('건강검진', 'Adverb')]
['건강검진']
054에 260에 8188로 전화하면 돼요
조금만 풀어서 설명해주시겠어요
